# Quantum Reservoir Computing (QRC) Steps
- Create a quantum reservoir.
- Encoding classical data into the reservoir.
- Extracting features and training a readout layer.


In [2]:
pip install qiskit-aer==0.11.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 16.1 MB/s eta 0:00:00


In [3]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Step 1: Create a Quantum Reservoir
A quantum reservoir is represented by a parameterized quantum circuit with random rotations and entanglement.


In [4]:
def create_quantum_reservoir(num_qubits, depth):
    """
    A quantum reservoir with random rotations and entanglement.
    :param num_qubits: Number of qubits in the reservoir.
    :param depth: Number of layers in the reservoir circuit.
    :return: QuantumCircuit object.
    """
    qc = QuantumCircuit(num_qubits)
    np.random.seed(42)  # For reproducibility

    for _ in range(depth):
        for qubit in range(num_qubits):
            qc.rx(np.random.uniform(0, 2 * np.pi), qubit)  # Random X-rotation
            qc.rz(np.random.uniform(0, 2 * np.pi), qubit)  # Random Z-rotation
        for qubit in range(num_qubits - 1):
            qc.cx(qubit, qubit + 1)  # Add entanglement
    return qc


## Step 2: Encode Input Data
Encode classical input data into the quantum reservoir using rotations.

In [5]:
def encode_input(qc, data, num_qubits):
    """
    Encode classical data into the quantum circuit.
    :param qc: QuantumCircuit object.
    :param data: List of input values.
    :param num_qubits: Number of qubits in the reservoir.
    """
    for i, value in enumerate(data):
        qc.ry(value, i % num_qubits)  # Encode data using Y-rotation


## Step 3: Run the Reservoir and Measure Outputs
Simulate the quantum reservoir and measure the qubit states.

In [6]:
def run_reservoir(qc, num_qubits):
    """
    Simulate the quantum reservoir and extract features from measurements.
    :param qc: QuantumCircuit object.
    :param num_qubits: Number of qubits.
    :return: Feature vector (measurement probabilities).
    """
    backend = Aer.get_backend('qasm_simulator')
    qc.measure_all()  # Add measurement gates
    result = execute(qc, backend, shots=100).result()  # Simulate with 100 shots
    counts = result.get_counts()

    # Normalize counts to get probabilities
    probabilities = []
    for i in range(2 ** num_qubits):
        state = bin(i)[2:].zfill(num_qubits)  # Convert index to binary string
        probabilities.append(counts.get(state, 0) / 100)
    return probabilities

## Step 4: Train and Evaluate the Linear Regression Model
Use the extracted features from the reservoir to train a linear regression model for classification.


In [7]:
# Parameters
num_qubits = 3
depth = 5
num_samples = 100

# Generate synthetic dataset
np.random.seed(42)
data = np.random.rand(num_samples, num_qubits)  # Random inputs
labels = (np.sum(data, axis=1) > 1.5).astype(int)  # Binary labels based on threshold

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Extract features from the quantum reservoir
def extract_features(data, num_qubits, depth):
    features = []
    for sample in data:
        qc = create_quantum_reservoir(num_qubits, depth)
        encode_input(qc, sample, num_qubits)
        features.append(run_reservoir(qc, num_qubits))
    return np.array(features)

# Extract features for training and testing
X_train_features = extract_features(X_train, num_qubits, depth)
X_test_features = extract_features(X_test, num_qubits, depth)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train_features, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_features)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert to binary predictions
accuracy = accuracy_score(y_test, y_pred_binary)

print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 95.00%
